
### Import Libraries

In [2]:
import numpy as np
import random
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

### Dataset

In [3]:
california_housing = fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

### Train-Test Split

In [4]:
def train_test_split(X,y,test=0.2):

    train_size = X.shape[0]-int(X.shape[0]*test)
    permutations=np.random.permutation(X.shape[0])

    X=X[permutations]
    y=y[permutations]

    X_train = X[:train_size]
    y_train = y[:train_size]
    X_test = X[train_size:]
    y_test = y[train_size:]

    return X_train, X_test, y_train, y_test

In [5]:
scaler=StandardScaler()
X_normalized=scaler.fit_transform(california_housing["data"])
X_train, X_test, y_train, y_test = train_test_split(X_normalized,california_housing["target"].reshape(-1,1),0.2)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(16512, 8)
(4128, 8)
(16512, 1)
(4128, 1)


### Neural Network

In [7]:
import numpy as np

class NeuralNetwork:
    def __init__(self, num_inputs=8,hidden_layers=10, learning_rate=0.001):
        self.lr = learning_rate
        self.w1 = np.random.randn(hidden_layers, num_inputs) * 0.01
        self.b1 = np.zeros((hidden_layers,1))
        self.w2 = np.random.randn(1,hidden_layers)*0.01
        self.b2 = np.zeros((1, 1))
    
    def model(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def ReLU_foward(self, x):
        return np.maximum(0, x),(x>0).astype(int)

    
    def affine_forward(self,x,w,b):
        out=np.dot(x,w.T)+np.repeat(b.T,1,axis=0)
        return out,w,x,np.ones(x.shape[0])

    def affine_backward(self,ud,ldx,ldw,ldb):
        db=np.sum(ud,axis=0)
        dw=np.dot(ud.T,ldw)
        dx=np.dot(ud,ldx)

        return dx,db,dw
    
    def loss_function(self,predicted,actual):
        loss=1/2*np.sum((predicted-actual)**2)
        dloss=predicted-actual
        return loss,dloss

    def gradient_descent(self):

        out1,ldx1,ldw1,ldb1=self.affine_forward(self.X_train,self.w1,self.b1)
        a_out1,lda_out1=self.ReLU_foward(out1)
        out2,ldx2,ldw2,ldb2=self.affine_forward(a_out1,self.w2,self.b2)

        self.Loss,dloss=self.loss_function(out2,self.y_train)
 
        dx2,db2,dw2=self.affine_backward(dloss,ldx2,ldw2,ldb2)
        da_out1= dx2 * (lda_out1>0)
        dx1,db1,dw1=self.affine_backward(da_out1,ldx1,ldw1,ldb1)


        db1=db1.reshape(-1,1)
        db2=db2.reshape(-1,1)

        self.w2 -= self.lr*dw2
        self.b2 -= self.lr*db2
        self.w1 -= self.lr*dw1
        self.b1 -= self.lr*db1

        return out2
    
    def output(self):
        out1,ldx1,ldw1,ldb1=self.affine_forward(self.X_train,self.w1,self.b1)
        a_out1,lda_out1=self.ReLU_foward(out1)
        out2,ldx2,ldw2,ldb2=self.affine_forward(a_out1,self.w2,self.b2)
        return out2

    def loss(self):
        return self.Loss

### Training Neural Network

#### Setting Parameters

In [8]:
num_inputs=8
hiddern_layers=32
learning_rate=0.0001
epochs=10
batch_size=1

In [9]:
NN=NeuralNetwork(num_inputs=num_inputs,hidden_layers=hiddern_layers,learning_rate=learning_rate)

#### Training

In [10]:
for epoch in range(0,epochs):
    for batch in range(0,X_train.shape[0],batch_size):
        NN.model(X_train[batch].reshape(1,-1),y_train[batch].reshape(1,-1))

        output=NN.gradient_descent()
        
        if batch%1000==0:
            print("Predicted : ",output,"   Actual : ",y_train[batch].reshape(1,-1))
            print("Loss : ",NN.loss())
            print()

Predicted :  [[0.00158623]]    Actual :  [[1.231]]
Loss :  0.7557291031019293

Predicted :  [[0.19253711]]    Actual :  [[3.937]]
Loss :  7.010501149812883

Predicted :  [[0.37116473]]    Actual :  [[2.769]]
Loss :  2.874806985177635

Predicted :  [[0.53399981]]    Actual :  [[2.706]]
Loss :  2.3587924147551522

Predicted :  [[0.67908544]]    Actual :  [[1.038]]
Loss :  0.06440982948934298

Predicted :  [[0.81684452]]    Actual :  [[1.597]]
Loss :  0.3043212858715531

Predicted :  [[0.93294178]]    Actual :  [[0.92]]
Loss :  8.374483795416068e-05

Predicted :  [[1.04179214]]    Actual :  [[1.18]]
Loss :  0.009550706067845716

Predicted :  [[1.15214412]]    Actual :  [[1.621]]
Loss :  0.10991291839032466

Predicted :  [[1.23640161]]    Actual :  [[1.024]]
Loss :  0.02255722155432638

Predicted :  [[1.32044274]]    Actual :  [[1.663]]
Loss :  0.058672738402815526

Predicted :  [[1.40268581]]    Actual :  [[2.]]
Loss :  0.17839212192844842

Predicted :  [[1.47776298]]    Actual :  [[1.299

### Testing Neural Network

#### Predicting Values

In [11]:
loss=[]
for batch in range(0,X_test.shape[0],batch_size):
    NN.model(X_test[batch].reshape(1,-1),y_test[batch].reshape(1,-1))
    output=NN.output()

    print("Predicted : ",output,"   Actual : ",y_train[batch].reshape(1,-1))

Predicted :  [[3.30425473]]    Actual :  [[1.231]]
Predicted :  [[1.27084454]]    Actual :  [[2.596]]
Predicted :  [[2.14903615]]    Actual :  [[1.245]]
Predicted :  [[1.57636939]]    Actual :  [[1.668]]
Predicted :  [[1.58126021]]    Actual :  [[5.00001]]
Predicted :  [[3.15640958]]    Actual :  [[2.848]]
Predicted :  [[2.21690197]]    Actual :  [[2.317]]
Predicted :  [[1.8353923]]    Actual :  [[0.685]]
Predicted :  [[3.56614452]]    Actual :  [[1.596]]
Predicted :  [[1.70828844]]    Actual :  [[2.357]]
Predicted :  [[2.23968618]]    Actual :  [[4.808]]
Predicted :  [[1.6151253]]    Actual :  [[1.601]]
Predicted :  [[1.56997472]]    Actual :  [[1.226]]
Predicted :  [[1.19028039]]    Actual :  [[1.996]]
Predicted :  [[2.33795997]]    Actual :  [[1.969]]
Predicted :  [[3.19454141]]    Actual :  [[5.00001]]
Predicted :  [[1.55261683]]    Actual :  [[3.414]]
Predicted :  [[1.47181669]]    Actual :  [[2.292]]
Predicted :  [[1.57833142]]    Actual :  [[2.12]]
Predicted :  [[1.87045007]]   